In [92]:
using LinearAlgebra, Test

# Matrix/Vector storage

In [38]:
x = UInt8[4,5,6]

3-element Vector{UInt8}:
 0x04
 0x05
 0x06

In [76]:
x[1] = 2 # converts 2 to UInt8, changes data x points to
x

3-element Vector{UInt8}:
 0x02
 0x05
 0x06

In [39]:
eltype(x), length(x), pointer(x) # these three things define a vector

(UInt8, 3, Ptr{UInt8} @0x0000000118e007f8)

In [4]:
p = pointer(x) #0x… is address in memory

Ptr{UInt8} @0x0000000118f8c678

In [5]:
# DONT DO THIS!
Base.unsafe_load(p)

0x04

In [14]:
Base.unsafe_load(p+2)

0x06

In [15]:
x = Int16[4,5,6]

3-element Vector{Int16}:
 4
 5
 6

In [18]:
p = pointer(x)
Base.unsafe_load(p)

4

In [21]:
T = eltype(x) # typeof(x[1])
Base.unsafe_load(p+sizeof(T))

5

In [26]:
x = [3.4, 5.6, 7.9]
p = pointer(x)
Base.unsafe_load(p + 8)

5.6

In [27]:
A = [1 2; 3 4; 5 6]

3×2 Matrix{Int64}:
 1  2
 3  4
 5  6

In [28]:
vec(A) # storage is same as a vector

6-element Vector{Int64}:
 1
 3
 5
 2
 4
 6

In [33]:
p = pointer(A)
T = eltype(A)
Base.unsafe_load(p + 3sizeof(T))

2

In [36]:
eltype(A), size(A), pointer(A) # these three things define a matrix

(Int64, (3, 2), Ptr{Int64} @0x0000000117ea5a80)

## Matrix*vector

In [52]:
function mul_rows(A, x)
    T = promote_type(eltype(A), eltype(x)) # "bigger" type, e.g. Float64
    m,n = size(A) # dimensions of matrix
    b = zeros(T, m) # we will return this
    for k = 1:m # do this for each row
        for j = 1:n # going along each column
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end

function mul_cols(A, x)
    T = promote_type(eltype(A), eltype(x)) # "bigger" type, e.g. Float64
    m,n = size(A) # dimensions of matrix
    b = zeros(T, m) # we will return this
    for j = 1:n # going along each column
        for k = 1:m # do this for each row
            b[k] = b[k] + A[k,j]*x[j]
        end
    end
    b
end

mul_cols (generic function with 1 method)

In [60]:
n = 5000
A = randn(n,n)
b = randn(n)

@time A*b;
@time mul_rows(A,b);
@time mul_cols(A,b); # 7x faster!

  0.007604 seconds (2 allocations: 39.109 KiB)
  0.140324 seconds (2 allocations: 39.109 KiB)
  0.020116 seconds (2 allocations: 39.109 KiB)


# Triangular matrices

In [63]:
A = [1 2  3; 4 5 6; 7 8 9]
U = UpperTriangular(A) # a subtype of AbstractMatrix, wraps A

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 1  2  3
 ⋅  5  6
 ⋅  ⋅  9

In [71]:
U[1,1] = 3

3

In [74]:
LowerTriangular(A)

3×3 LowerTriangular{Int64, Matrix{Int64}}:
 3  ⋅  ⋅
 4  5  ⋅
 7  8  9

In [73]:
A

3×3 Matrix{Int64}:
 3  2  3
 4  5  6
 7  8  9

In [77]:
# UpperTriangular can use only n*(n-1)/2 entries:
function mul_cols(U::UpperTriangular, x)
    T = promote_type(eltype(U), eltype(x)) # "bigger" type, e.g. Float64
    m,n = size(U) # dimensions of matrix
    b = zeros(T, m) # we will return this
    for j = 1:n # going along each column
        for k = 1:j # do this for each row
            b[k] = b[k] + U[k,j]*x[j]
        end
    end
    b
end

mul_cols (generic function with 2 methods)

In [95]:
n = 1000
A = randn(n,n)
U= UpperTriangular(A)
b = randn(n)


@time A*b;
@time U*b;
@time mul_cols(A,b); 
@time mul_cols(U,b);

  0.000457 seconds (1 allocation: 7.938 KiB)
  0.000308 seconds (1 allocation: 7.938 KiB)
  0.000673 seconds (1 allocation: 7.938 KiB)
  0.000579 seconds (1 allocation: 7.938 KiB)


In [91]:
mul_cols(U,b) ≈ U*b

true

In [108]:
# UpperTriangular can use only n*(n-1)/2 entries:
function ldiv(U::UpperTriangular, b)
    T = promote_type(eltype(U), eltype(b)) # "bigger" type, e.g. Float64
    m,n = size(U) # dimensions of matrix
    # we know m == n
    x = zeros(T, m) # we will return this
    for k = n:-1:1
        r = b[k]
        for j = 2:n
            r = r - A[k,j]*x[j]
        end
        x[k] = r/U[k,k]
    end
    x
end

ldiv (generic function with 1 method)

In [111]:
n = 1000
A = randn(n,n) + 100I # WHY???
U = UpperTriangular(A)
x = randn(n)


b = U*x
ldiv(U,b) ≈ x

true

## Banded matrices

In [123]:
d = [1,2,3,4,5]
D = Diagonal(d) # stores a vector

D[3,3] = 10

10

In [131]:
d = [1,2,3,4,5]
u = [5,6,7,8]
U = Bidiagonal(d, u, :U) # stores 2 vectors and U.uplo. Fast multiplication and inv:

fieldnames(typeof(U))
U.uplo

'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)

In [174]:
function ldiv(U::Bidiagonal, b)
    T = promote_type(eltype(U), eltype(b)) # "bigger" type, e.g. Float64
    m,n = size(U) # dimensions of matrix
    # we know m == n
    x = zeros(T, m) # we will return this
    
    if U.uplo == 'U'
        for k = n:-1:1
            r = b[k]
            if k ≠ n
                r = r - U[k,k+1]*x[k+1]
            end
            x[k] = r/U[k,k]
        end
    else
        error("Lower not implemented")
    end
    x
end

ldiv (generic function with 2 methods)

In [177]:
x = randn(5)
b = U*x
ldiv(U, b) ≈ x

true

In [178]:
d = [1,2,3,4,5]
u = [5,6,7,8]
l = [9,10,11,12]
T = Tridiagonal(l, d, u)

5×5 Tridiagonal{Int64, Vector{Int64}}:
 1   5   ⋅   ⋅  ⋅
 9   2   6   ⋅  ⋅
 ⋅  10   3   7  ⋅
 ⋅   ⋅  11   4  8
 ⋅   ⋅   ⋅  12  5